In [3]:
!pip install transformers datasets wandb pycocotools


Defaulting to user installation because normal site-packages is not writeable


In [4]:
# DI725 - Assignment 2: Object Detection with Hugging Face DETR + AU-AIR + WANDB
import os
import json
import torch
import wandb
from PIL import Image
from torch.utils.data import Dataset, DataLoader
from transformers import DetrImageProcessor, DetrForObjectDetection
from pycocotools.coco import COCO
from pycocotools.cocoeval import COCOeval

C:\Users\nesil.bor\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.12_qbz5n2kfra8p0\LocalCache\local-packages\Python312\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [ ]:
# 🔧 Paths
root_dir = r"C:\Users\nesil.bor\Desktop\Folders\master\DI725\DI725_Assignment2_2030336\data\auair2019"
annotation_path = os.path.join(root_dir, "annotations.json")

# ⚙️ Init WANDB
wandb.init(project="di725-assignment2", name="huggingface-detr-auair")

# 🧠 Load Processor + Model
processor = DetrImageProcessor.from_pretrained("facebook/detr-resnet-50")
model = DetrForObjectDetection.from_pretrained("facebook/detr-resnet-50").to("cuda" if torch.cuda.is_available() else "cpu")

# 📦 AU-AIR Dataset Class
class AUAIRDetrDataset(Dataset):
    def __init__(self, root, annotation_file):
        with open(annotation_file, 'r') as f:
            data = json.load(f)
        self.annotations = data['annotations']
        self.categories = data['categories']
        self.img_dir = os.path.join(root, "images")

    def __len__(self):
        return len(self.annotations)

    def __getitem__(self, idx):
        ann = self.annotations[idx]
        img_path = os.path.join(self.img_dir, ann["image_name"])
        image = Image.open(img_path).convert("RGB")

        boxes, class_labels = [], []
        annotations = []

        for bbox in ann["bbox"]:
            x, y, w, h = bbox["left"], bbox["top"], bbox["width"], bbox["height"]
            area = w * h
            category_id = bbox["class"] + 1  # +1 for DETR
            
            boxes.append([x, y, x + w, y + h])
            class_labels.append(category_id)
            
            annotations.append({
                "bbox": [x, y, w, h],
                "category_id": category_id,
                "area": area,
                "iscrowd": 0
                 })

        encoding = processor(images=image, annotations={
            "image_id": idx,
            "annotations": annotations
        }, return_tensors="pt")

        pixel_values = encoding["pixel_values"].squeeze()
        target = encoding["labels"][0]

        return pixel_values, target

# 📋 Load dataset
dataset = AUAIRDetrDataset(root_dir, annotation_path)
loader = DataLoader(dataset, batch_size=2, shuffle=True, collate_fn=lambda x: tuple(zip(*x)))

# 🧪 Training
device = "cuda" if torch.cuda.is_available() else "cpu"
optimizer = torch.optim.AdamW(model.parameters(), lr=5e-5)
model.train()

for epoch in range(5):
    total_loss = 0
    for pixel_values, targets in loader:
        pixel_values = torch.stack(pixel_values).to(device)
        labels = [{k: v.to(device) for k, v in t.items()} for t in targets]

        outputs = model(pixel_values=pixel_values, labels=labels)
        loss = outputs.loss
        loss.backward()
        optimizer.step()
        optimizer.zero_grad()

        total_loss += loss.item()

    avg_loss = total_loss / len(loader)
    print(f"Epoch {epoch+1} - Loss: {avg_loss:.4f}")
    wandb.log({"epoch": epoch+1, "loss": avg_loss})

# 💾 Save model
model.save_pretrained("hf_detr_auair")
processor.save_pretrained("hf_detr_auair")
wandb.save("hf_detr_auair/*")



Some weights of the model checkpoint at facebook/detr-resnet-50 were not used when initializing DetrForObjectDetection: ['model.backbone.conv_encoder.model.layer1.0.downsample.1.num_batches_tracked', 'model.backbone.conv_encoder.model.layer2.0.downsample.1.num_batches_tracked', 'model.backbone.conv_encoder.model.layer3.0.downsample.1.num_batches_tracked', 'model.backbone.conv_encoder.model.layer4.0.downsample.1.num_batches_tracked']
- This IS expected if you are initializing DetrForObjectDetection from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing DetrForObjectDetection from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


KeyError: 'area'

In [ ]:
# 📈 COCO-style Evaluation (same as before)
def prepare_coco_format(dataset, output_path="gt.json"):
    coco_dict = {"images": [], "annotations": [], "categories": []}
    ann_id = 1
    for idx, ann in enumerate(dataset.annotations):
        img_id = idx + 1
        coco_dict["images"].append({
            "id": img_id,
            "file_name": ann["image_name"],
            "width": ann["image_width:"],
            "height": ann["image_height"]
        })
        for box in ann["bbox"]:
            coco_dict["annotations"].append({
                "id": ann_id,
                "image_id": img_id,
                "category_id": box["class"] + 1,
                "bbox": [box["left"], box["top"], box["width"], box["height"]],
                "area": box["width"] * box["height"],
                "iscrowd": 0
            })
            ann_id += 1
    for i, cat in enumerate(dataset.categories):
        coco_dict["categories"].append({"id": i + 1, "name": cat})
    with open(output_path, 'w') as f:
        json.dump(coco_dict, f)
    return output_path

# 🔍 Inference + Prediction Format
def run_inference_and_save_predictions(model, dataset, output_path="pred.json"):
    model.eval()
    results = []
    for idx in range(len(dataset)):
        image, _ = dataset[idx]
        inputs = processor(images=image, return_tensors="pt").to(device)

        with torch.no_grad():
            outputs = model(**inputs)

        target_sizes = torch.tensor([[480, 480]]).to(device)
        results_ = processor.post_process_object_detection(outputs, target_sizes=target_sizes, threshold=0.5)[0]

        for i in range(len(results_["scores"])):
            box = results_["boxes"][i]
            score = results_["scores"][i].item()
            label = results_["labels"][i].item()
            xmin, ymin, xmax, ymax = box
            results.append({
                "image_id": idx + 1,
                "category_id": label,
                "bbox": [float(xmin), float(ymin), float(xmax - xmin), float(ymax - ymin)],
                "score": float(score)
            })

    with open(output_path, 'w') as f:
        json.dump(results, f)

    return output_path

# 🧮 mAP Evaluation
def evaluate_map(gt_path, pred_path):
    coco_gt = COCO(gt_path)
    coco_dt = coco_gt.loadRes(pred_path)
    coco_eval = COCOeval(coco_gt, coco_dt, iouType='bbox')
    coco_eval.evaluate()
    coco_eval.accumulate()
    coco_eval.summarize()

    metrics = {
        "AP (IoU=0.50:0.95)": coco_eval.stats[0],
        "AP50": coco_eval.stats[1],
        "AP75": coco_eval.stats[2],
        "AP_small": coco_eval.stats[3],
        "AP_medium": coco_eval.stats[4],
        "AP_large": coco_eval.stats[5]
    }

    wandb.log(metrics)
    print("✅ Evaluation metrics logged to WANDB.")
    return metrics

# 📊 Run Full Eval
gt_json = prepare_coco_format(dataset)
pred_json = run_inference_and_save_predictions(model, dataset)
evaluate_map(gt_json, pred_json)

# ✅ Done
wandb.finish()